# Total Cost of Ownership (TCO) Analysis

Last Updated on January 25, 2019

### *** To start off, select the first cell with Python code below and then press Shift + Enter to run the cell. You can continue to press Shift + Enter as the selection moves forward to the next cell. 

## Import modules

In [ ]:
from datetime import datetime as dt
from IPython.display import display, HTML
from ipywidgets import widgets, interactive, Layout
import numpy as np
import pandas as pd
import qgrid
import my_functions as my
import my_DB_functions as myDB

pd.options.display.float_format = '{:,.2f}'.format

## Define variables

In [ ]:
USER_ID = ! whoami
USER_ID = USER_ID[0]
MAX_YEARS = 5

## Step 1: Load available data

In [ ]:
records = myDB.get_avail_records(USER_ID)
df = pd.DataFrame(records, columns=["ID", "UserID", "UploadTime", "Org", "System", "Description", "Type",
                                    "DeployDate", "Measure", "X", "Y", "BenchmarkDate",  "Availability"])
df = df.set_index("ID")
df.Availability = df.Availability.apply(lambda x:"Public" if x == 1 else "Private")

if len(df) == 0:
    my.print_title("No available data found.", "h3")
    my.print_title("Please input your data first by using manage_data.ipynb.", "h3")

## Step 2: Select records of interest
### Notice that if you would want to input new data, use manage_data.ipynb. 

In [ ]:
my.print_title("Select one or more of the available records." + 
               "You can select multiple rows by pressing and holding the Ctrl key as you click a record.", "h3")
q = qgrid.show_grid(df.drop(["UploadTime"], axis=1))
q

## Display the selected records

In [ ]:
my.print_title("Benchmark Data", "h2")

if len(q.get_selected_rows()) == 0:
    q.change_selection([1, 2, 3])

record_id_list = sorted(q.get_selected_df().index)
df_xy_list = []

for record_id in record_id_list:
    user_id = df["UserID"][record_id]
    ts = df["UploadTime"][record_id]
    org_name = df["Org"][record_id]
    sys_name = df["System"][record_id]
    sys_type = df["Type"][record_id]
    x = df["X"][record_id]
    y = df["Y"][record_id]

    cursor = myDB.get_selected_benchmark_records(user_id, ts)
    df_xy = pd.DataFrame(list(cursor), columns=["x", "y"])
    df_xy_list.append(df_xy)

    my.print_title("{} {} {} (ID: {})".format(org_name, sys_name, sys_type, record_id), "h4")
    print("x: {}, y: {}".format(x, y))
    display(df_xy)

## Step 3: Display the parameters for TCO calculation
### Notice that you can edit the parameter values if you would want. The changes made here will not be saved. 

In [ ]:
my.print_title("TCO Parameters", "h2")

widgets_list = []

style = {"description_width": '300px'}

for record_id in record_id_list:
    user_id = df["UserID"][record_id]
    ts = df["UploadTime"][record_id]
    org_name = df["Org"][record_id]
    sys_name = df["System"][record_id]
    sys_type = df["Type"][record_id]
        
    my.print_title("{} {} {} (ID {})".format(org_name, sys_name, sys_type, record_id), "h4")
    
    cursor = myDB.get_tco_questions(sys_type)
    cursor2 = myDB.get_tco_data(sys_type, user_id, ts)
    
    tco_values = list(cursor2)[0]
    
    tmp = {}
    
    for item, tco_value in zip(list(cursor), tco_values[3:]):
        question = item[0]
        variable_name = item[1]
        unit = item[2]

        w = widgets.FloatText(
            description = "{} {}".format(question, "" if unit == "" else "({})".format(unit)),
            value = tco_value,
            layout = Layout(width="45%"),
            style = style
        )
        display(w)

        tmp[variable_name] = w

    widgets_list.append(tmp)

In [ ]:
params_list = []

for item, record_id in zip(widgets_list, record_id_list):
    org_name = df["Org"][record_id] 
    sys_name = df["System"][record_id]
    sys_type = df["Type"][record_id] 
    
    params = {}
    params["org_name"] = org_name
    params["sys_name"] = sys_name
    params["sys_type"] = sys_type
    for key in item.keys():
        w = item[key]
        params[key] = w.value
        
    params_list.append(params)

## Step 4: Display the performance scaling functions

In [ ]:
my.print_title("Performance Scaling Functions", "h2")

style = {'description_width': '200px'}

params = {}

for record_id in record_id_list:
    w_degree = widgets.IntSlider(
        value = 2,
        min = 1,
        max = 3,
        step = 1,
        description = "Degree of Polynomial for ID {}".format(record_id),
        layout = Layout(width="50%"),
        style = style
    )
    
    params["degree_{}".format(record_id)] = w_degree

def view(**params):
    display(HTML("<hr>"))
    my.display_multiple_scaling(df, record_id_list, df_xy_list, **params)

interactive(view, **params)

## Step 5: Analyze and compare TCOs

In [ ]:
print("\n")
style = {'description_width': '150px'}
layout = Layout(width="70%")

w_years = widgets.IntSlider(
    description = "Life Span (Years)",
    min = 0,
    max = MAX_YEARS,
    step = 1,
    value = 5,
    style = style,
    layout = layout
)

w_sys_util = widgets.IntSlider(
    description = "System Utilization (%)",
    min = 0,
    max = 100,
    step = 5,
    value = 100,
    style = style, 
    layout = layout
)

nums_servers = {}
for i in range(len(params_list)):
    params = params_list[i]
    
    sys_type = params["sys_type"]
    sys_name = params["sys_name"]
    num_servers = params["instance_count"] if sys_type == "Cloud" else params["node_count"]
    w_num_servers = widgets.IntText(
        description = "# of {} for {}".format("Instances" if sys_type == "Cloud" else "Nodes", sys_name),
        value = num_servers,
        layout = Layout(width="40%"),
        style = style
    )
    nums_servers["num_servers_{}".format(i)] = w_num_servers
    
def view(years, util, **nums_servers):
    my.display_compare_TCO(params_list, years, util, df, record_id_list, df_xy_list, **nums_servers)

interactive(view, years=w_years, util=w_sys_util, **nums_servers)